### Entropy


Intuitively, we can think of **entropy** as the measure of disorder in a system. 

This set, $S$, for example, is very disordered:

<img src="https://www.evernote.com/l/AAEZbuSslJRCwKxfZaSIBJaIZqcQ-mTe9RQB/image.png" width=400px>


Let us consider the elements of $S$ that have a black border.

In [ ]:
%run items.py

In [ ]:
import numpy as np
import pandas as pd

from IPython.display import display

items_df = pd.DataFrame(items)
items_df

## Information Entropy


We can measure the disorder in $S$ relative to any attribute of an element using the Shannon Information **Entropy**

$$H(S) = -p_1\log_2p_1 + \dots + -p_n\log_2p_n$$




Here, each $p_i$ is a measure of the proportion of the set represented by each class for a given attribute. If we are looking at the color of our shapes, this would be five classes: 

In [ ]:
items_df.color.unique()

We can use Python to calculate the entropy

In [ ]:
def class_entropy(proportion):
    return -proportion*np.log2(proportion)

In [ ]:
def entropy(proportions):
    class_entropies = [class_entropy(proportion) 
                       for proportion in proportions]
    return sum(class_entropies)

### Measure The Entropy of the color of $S$ 

In [ ]:
n = items_df.color.count()
n

In [ ]:
p_red = (items_df.color == 'red').sum()/n
p_blue = (items_df.color == 'blue').sum()/n
p_green = (items_df.color == 'green').sum()/n
p_silver = (items_df.color == 'silver').sum()/n
p_purple = (items_df.color == 'purple').sum()/n

S_proportions = [p_red, p_blue, p_green, p_silver, p_purple]
S_proportions

In [ ]:
sum(S_proportions)

In [ ]:
entropy(S_proportions)

## To Do

Write a method that calculates the class proportions for a given dataframe and feature.


In [ ]:
def class_proportions(series):
    classes = series.unique()
    total_count = series.count()
    counts = [(series == cls).sum() for cls in classes]
    return counts/total_count

In [ ]:
class_proportions(items_df.color)

In [ ]:
def entropy(series):
    proportions = class_proportions(series)
    class_entropies = [class_entropy(proportion) 
                       for proportion in proportions]
    return sum(class_entropies)

Use this method to calculate the Entropy with respect to each attribute of $S$

In [ ]:
display("color:  {}".format(class_proportions(items_df.color)))
display("form:   {}".format(class_proportions(items_df.form)))
display("letter: {}".format(class_proportions(items_df.letter)))
display("border: {}".format(class_proportions(items_df.border)))

In [ ]:
display("color:  {}".format(entropy(items_df.color)))
display("form:   {}".format(entropy(items_df.form)))
display("letter: {}".format(entropy(items_df.letter)))
display("border: {}".format(entropy(items_df.border)))

## The Decision Tree

Decision Trees are supervised learning models typically split into classification trees and regression trees. For the rest of this lesson, we will focus on classification trees. 

We will work through the ID3 algorithm for learning a decision tree from a set of labeled data. 

#### Labeled Data

For our purposes, let us assume that the feature `border` is our label. We will be seeking a decision tree that makes splits in order to develop a model for identifying which items will have a border. 

In [ ]:
features = items_df.drop('border', axis=1)
target = items_df.border

In [ ]:
class_proportions(target)

In [ ]:
entropy(target)

### Three Partitioning Schemes

We can start by separating the elements based upon their attributes. Here are three different ways to do that:


![](img/entropy_3.png)

### Identify the Best Split

In order to proceed, we will need to identify which of these ways of separating is best? We can use the measure of entropy to do this!


#### Split by Form

In [ ]:
features_square_df = features[features.form == 'square']
features_circle_df = features[features.form == 'circle']
features_diamond_df = features[features.form == 'diamond']
features_star_df = features[features.form == 'star']

target_square_df = target[features.form == 'square']
target_circle_df = target[features.form == 'circle']
target_diamond_df = target[features.form == 'diamond']
target_star_df = target[features.form == 'star']

display(pd.merge(features_square_df, pd.DataFrame(target_square_df), left_index=True, right_index=True))
display(pd.merge(features_circle_df, pd.DataFrame(target_circle_df), left_index=True, right_index=True))
display(pd.merge(features_diamond_df, pd.DataFrame(target_diamond_df), left_index=True, right_index=True))
display(pd.merge(features_star_df, pd.DataFrame(target_star_df), left_index=True, right_index=True))


To assess this split, we will be seeking the entropy associated with each subset as a proportion of the total set.

$$H_T = q_{\text{square}}H(S_{\text{square}}) + 
        q_{\text{circle}}H(S_{\text{circle}}) + 
        q_{\text{diamond}}H(S_{\text{diamond}}) + 
        q_{\text{star}}H(S_{\text{star}})
$$

Here each $q_i$ represents the **weight**, the proportion of the total each subset is

$$q_i = \frac{\text{count}(S_i)}{\text{count}(S_T)}$$

In [ ]:
q_square = features_square_df.form.count()/features.form.count()
q_circle = features_circle_df.form.count()/features.form.count()
q_diamond = features_diamond_df.form.count()/features.form.count()
q_star = features_star_df.form.count()/features.form.count()

### Total Entropy When Split by Form

##### Calculate the weights of each class of our target

In [ ]:
display("color:  {}".format(class_proportions(target_square_df)))
display("form:   {}".format(class_proportions(target_circle_df)))
display("letter: {}".format(class_proportions(target_diamond_df)))
display("border: {}".format(class_proportions(target_star_df)))

##### Find the entropy for these proportions

In [ ]:
display("square:  {}".format(entropy(target_square_df)))
display("circle:   {}".format(entropy(target_circle_df)))
display("diamond: {}".format(entropy(target_diamond_df)))
display("star: {}".format(entropy(target_star_df)))

In [ ]:
H_square_border = entropy(target_square_df)
H_circle_border = entropy(target_circle_df)
H_diamond_border = entropy(target_diamond_df)
H_star_border = entropy(target_star_df)

##### Sum the weighted Entropy

In [ ]:
q_square*H_square_border + q_circle*H_circle_border + q_diamond*H_diamond_border + q_star*H_star_border

### Write a function to do this for a split on any feature

In [ ]:
def entropy_on_split(dataframe, target, feature, debug=False):
    
    # split on feature
    unique_classes = dataframe[feature].unique()    
    target_subsets = [
        target[dataframe[feature] == unique_class] 
        for unique_class in unique_classes
    ]
        
    # calculate subset weights
    total = target.count()
    weights = [
        target_subset.count()/total
        for target_subset in target_subsets
    ]
    
    # calculate entropies
    entropies = [
        entropy(target_subset)
        for target_subset in target_subsets
    ]
        
    # return weighted entropy
    return sum(weight*entropy for weight, entropy in zip(weights, entropies))

In [ ]:
entropy_on_split(features, target, 'form')

In [ ]:
entropy_on_split(features, target, 'color')

In [ ]:
entropy_on_split(features, target, 'letter')

### Write a function to Identify the Best Split

In [ ]:
def find_best_split(features, target):
    feature_labels = features.columns
    entropies = [
        entropy_on_split(features, target, feature_label)
        for feature_label in feature_labels
    ]
    
    best_index = np.argmin(entropies)
    return feature_labels[best_index]

In [ ]:
find_best_split(features, target)

## Representing the Tree

The ability to find a best split will only work at a single node. In order to build a complete machine learning model, we are going to need to use this method to build an entire decision tree.

We have this so far:

<img src="https://www.evernote.com/l/AAFzXwQ0IPVAcYpRMWQuaQCf0NvtOC4mz0sB/image.png" width=400px>

But what about the rest of the tree? We want to split like this:

<img src="https://www.evernote.com/l/AAHMTpccFvtOjYGojh7sZ97oDC6-bRNAPe4B/image.png" width=400px>

<img src="https://www.evernote.com/l/AAGE0WSC6F5AzLCUX-A3cwXrQULf_uFRyFYB/image.png" width=400px>

In order to represent this using Python, we will define a `tree` to be one of these:

- `True`
- `False`
- a tuple `(attribute, subtree)`

For example, consider the tree representing green circles containing the letter 'B'.

We can represent this trivially as:

In [ ]:
green_circle_B_tree = False

or the tree representing green circles containing the letter 'D':

In [ ]:
green_circle_D_tree = True

From there, consider the tree representing green circles:

<img src="https://www.evernote.com/l/AAEbErEwS7hETI2gNXzkaetSXntpxeGKjSoB/image.png" width=200px>

In [ ]:
green_circle_tree = ('letter', {'B' : False, 'D' : True})

We might gradually work our way up from there and represent our entire tree as

In [ ]:
tree = ('form', {'square': False,
                 'circle': 
                           ('color', {'green': 
                                               ('letter', {'B' : False, 
                                                           'D' : True}),
                                      'red': True,
                                      'blue': True}),
                 'diamond': 
                            ('letter', {'C': False,
                                        'D': True,
                                        'E': True}),
                 'star': True})

### Use a tree to classifiy an input

Suppose we had a new element and we wish to no whether or not is has a border. For example, we may be given the following element:

    {'form': 'circle', 'letter': 'C', 'color': 'red'}

We are going to need to build a classification function to use our tree to classify this input.

In [ ]:
def classify(tree, element):
    if tree in [True, False]:
        return tree
    
    attribute, subtree_dictionary = tree
    
    subtree_key = element.get(attribute)
    
    if subtree_key not in subtree_dictionary:
        subtree_key = None
    
    subtree = subtree_dictionary[subtree_key]
    
    return classify(subtree, element)

In [ ]:
classify(tree, {'form': 'circle', 'letter': 'C', 'color': 'red'})

But what if we pass an element that is ambiguous, for example

    {'form': 'circle', 'letter': 'C', 'color': 'green'}

In [ ]:
classify(tree, {'form': 'circle', 'letter': 'C', 'color': 'green'})

### To Handle this, we will redefine our Tree

We will add a `None` key that returns the most common class.

In [ ]:
tree = ('form', {'square': False,
                 'circle': 
                           ('color', {'green': 
                                               ('letter', {'B' : False, 
                                                           'D' : True,
                                                           None: False}),
                                      'red': True,
                                      'blue': True,
                                      None : True}),
                 'diamond': 
                            ('letter', {'C': False,
                                        'D': True,
                                        'E': True,
                                        None: True}),
                 'star': True,
                 None: True})

In [ ]:
def classify(tree, element):
    if tree in [True, False]:
        return tree
    
    attribute, subtree_dictionary = tree
    
    subtree_key = element.get(attribute)
    
    if subtree_key not in subtree_dictionary:
        subtree_key = None
    
    subtree = subtree_dictionary[subtree_key]
    
    return classify(subtree, element)

In [ ]:
classify(tree, {'form': 'circle', 'letter': 'C', 'color': 'green'})

In [ ]:
classify(tree, {'form': 'octagon', 'letter': 'Z', 'color': 'chartreuse'})

## Build The Tree

In [ ]:
def find_best_split(features, target, split_candidates):
        
    entropies = [
        entropy_on_split(features, target, feature_label)
        for feature_label in split_candidates
    ]
    
    best_index = np.argmin(entropies)
    return split_candidates[best_index]

In [ ]:
def build_tree(features, target, split_candidates=None):
    if split_candidates is None:
        split_candidates = list(features.columns)
    
    total_count = target.count()
    true_count = target.sum()
    false_count = total_count - true_count
    
    if false_count == 0: return True
    if true_count == 0: return False
    
    if split_candidates == []:
        return true_count > false_count
    
    best_attribute = find_best_split(features, target, split_candidates)

    split_candidates = [split_candidate 
                        for split_candidate in split_candidates 
                        if split_candidate is not best_attribute]
    
    best_attribute_uniques = features[best_attribute].unique()
    subtree = dict()
    for best_attr_unique in best_attribute_uniques:
        feat_subset = features[features[best_attribute] == best_attr_unique]
        target_subset = target[features[best_attribute] == best_attr_unique]
        subtree[best_attr_unique] = build_tree(feat_subset, target_subset, split_candidates)
    subtree[None] = true_count > false_count

    return (best_attribute, subtree)

In [ ]:
build_tree(features, target)

In [ ]:
my_tree = build_tree(features, target)

In [ ]:
display(classify(my_tree, {'form': 'circle', 'letter': 'C', 'color': 'red'}))
display(classify(my_tree, {'form': 'circle', 'letter': 'C', 'color': 'green'}))
display(classify(my_tree, {'form': 'octagon', 'letter': 'Z', 'color': 'chartreuse'}))